In [19]:
import requests 
import os
import pandas as pd

In [13]:
# Defining a function that will make an API call to Alpha Vantage for the TIME_SERIES_DAILY function

def query_data(symbol, api_key, outputsize='compact', datatype='json'):
    """
    Fetches intraday trading data for a stock ticker and interval.

    Args:
    - symbol (str): The ticker symbol of the stock (e.g. 'IBM').
    - api_key (str): API key for the Alpha Vantage API.
    - outputsize (str): Data output size ('compact' or 'full'). compact = latest 100 data points, full = full-length time series of 20+ years data. Defaults to 'compact'.
    - datatype (str): Format of the data returned ('json' or 'csv'). Defaults to 'json'.

    Returns:
    - dict or str: The data returned by the API in JSON or CSV depending on 'datatype'.
    """
    # API endpoint
    url = "https://www.alphavantage.co/query"
    
    # Parameters 
    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': symbol,
        'outputsize': outputsize,
        'datatype': datatype,
        'apikey': api_key
    }

    # Send the GET request to the API
    response = requests.get(url, params=params)

    # Check the request status
    if response.status_code == 200:
        if datatype == 'json':
            return response.json()  
        else:
            return response.text  
    else:
        raise Exception(f"Failed to retrieve data: {response.status_code}")

In [38]:
# Making the API call for the ticker 'SU' (Suncor Energy inc.)

api_key = os.environ.get('AlphaVantageAPIKey')
data = query_data('SU', api_key, outputsize='full')


In [41]:
# Defining a function that will convert JSON data into a dataframe

def extract_json(json_data):
    """
    Extracts and converts the 'Time Series (Daily)' portion of a JSON object into a pandas DataFrame.
    
    Args:
    - json_data (dict): JSON object containing the stock market data.
    
    Returns:
    - pd.DataFrame: DataFrame containing the daily time series data with dates as the index.
    """
    # Extract the 'Time Series (Daily)' part of the JSON data
    time_series_data = json_data.get('Time Series (Daily)', {})

    # Convert into a DataFrame
    df = pd.DataFrame.from_dict(time_series_data, orient='index')

    # Clean up the column names by removing numbers and periods
    df.columns = [col.split('. ')[1] for col in df.columns]
    
    # Convert the index to a datetime index
    df.index = pd.to_datetime(df.index)
    
    # Convert all columns to float type
    df = df.astype(float)
    
    return df

In [42]:
# Calling the function to convert the JSON into a dataframe

extract_json(data)

,open,high,low,close,volume
2024-04-17,37.68,38.280,37.345,37.89,3278957.0
2024-04-16,37.05,37.790,36.932,37.58,5897756.0
2024-04-15,37.68,37.910,37.035,37.18,4531106.0
2024-04-12,38.82,38.925,37.450,37.69,5330623.0
2024-04-11,39.31,39.310,38.250,38.36,4827506.0
...,...,...,...,...,...
1999-11-05,36.25,36.440,35.500,35.62,9400.0
1999-11-04,36.62,37.120,35.880,36.12,22500.0
1999-11-03,38.19,38.190,36.750,36.75,41700.0
1999-11-02,37.94,37.940,37.000,37.31,5800.0


In [43]:
# Saving the dataframe into a csv file

extract_json(data).to_csv('SU_SuncorEnergy.csv', index=True)